In [1]:
import pandas as pd
import matplotlib as mp
import numpy as np
from datetime import timedelta
import datetime as dt

from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering

%matplotlib inline

#Root de archivos de datos
root_dir = '../data/'

#Lee menos data si es testing
is_testing = False

#Nuecleos disponibles
cores = 4

# Cantidad minima de aparicion de equipos en un dataframe
min_devices = 5

# Identificador unico de equipos
device_uuid = ['ref_hash']

# Ventanas de tiempo
windows = pd.DataFrame({
    'begin_date': [dt.datetime(2019,4,18),dt.datetime(2019,4,21),dt.datetime(2019,4,24)],
    'window_nr':[1,2,3]
})

# Armado de Features

Se utilizan 3 ventanas de tiempo y se procesan los dataframes para obtener features de esas 3 ventanas por separado. 

Como resultado se obtiene un dataset de features mas grande cuyos datos no se solapan en tiempo. Por lo que son validos para la prediccion. 

Se realiza la lectura y limpiado de los dataframes principales, con los mismos se realiza el filtrado y armado de los features. 

---
## 1. Lectura de DataFrames

In [2]:
#Optimizado para menos memoria
auction_dtypes = {
    'device_id': np.int64,
    'source_id': np.int8
}

auctions = pd.read_csv(root_dir + 'auctions.csv.gzip',
                       compression = 'gzip',
                       dtype = auction_dtypes,
                       usecols=list(auction_dtypes.keys()) + ['date'],
                       parse_dates = ['date'])
auctions.rename({'device_id':'ref_hash',
                 'date':'created'}, inplace=True, axis='columns')
auctions['n'] = 1

# Para hacer pruebas ocupando menos memoria se hace un sampleo aleatorio de la mitad del dataframe y se elimina el resto
if is_testing:
    auctions = auctions.sample(frac=.30)
    print("Is Testing, #records:", auctions['n'].sum())
else:
    print("Not Testing, #records:", auctions['n'].sum())

Not Testing, #records: 47409528


In [3]:
# Labels a submitir con las predicciones
to_predict = pd.read_csv(root_dir + 'target_competencia_ids.csv')
to_predict = to_predict.apply(lambda x: np.int64(x['ref_hash'][0:x['ref_hash'].find('_')]), axis='columns').drop_duplicates().to_frame()
to_predict.columns = ['ref_hash']

In [4]:
unique_hashes = pd.read_csv(root_dir+'unique_hashes.csv')

---
## 2. Arreglo de los datos

### Separo las semanas de entrenamiento

Se utiliza el siguiente metodo: 

1. Ventana del 18 al 20 inclusive (1) -> Predice valores entre el 21 y 24 (2)
2. Ventana del 21 al 23 inclusive (2) -> Predice valores entre el 24 y 26 (3)

In [5]:
#Agrego la ventana de tiempo
auctions = auctions.loc[auctions['ref_hash'].isin(unique_hashes['ref_hash'])]\
                   .sort_values(by='created')
auctions = pd.merge_asof(auctions,windows,left_on='created',right_on='begin_date').drop('begin_date', axis='columns')

### 2.1 Arreglo de Auctions

In [6]:
%%time
#Elimino los registros con menos de un minimo de entradas, ya que no hay mucho que predecir en estos casos
group = ['ref_hash','window_nr']

orig_count = auctions['n'].sum()
auctions = auctions.groupby(group, sort = False)\
                   .filter(lambda x: x['n'].sum() >= min_devices)\
                   .sort_values(by=group+['created'])
last_count = auctions['n'].sum()
print('Eliminados:', orig_count-last_count)

Eliminados: 736875
CPU times: user 10min 8s, sys: 18.3 s, total: 10min 26s
Wall time: 10min 22s


In [7]:
%%time
auctions['next_date'] = auctions.groupby(group, as_index = False, sort=False)['created'].transform(lambda x: x.shift(-1))
auctions = auctions.loc[(~auctions['next_date'].isnull())]
auctions['secs_to_next'] = (auctions['next_date'] - auctions['created']).transform(lambda x: round(x.total_seconds()))

CPU times: user 17min 4s, sys: 17.1 s, total: 17min 21s
Wall time: 17min 19s


In [8]:
auctions.head()

,created,ref_hash,source_id,n,window_nr,next_date,secs_to_next
9146132,2019-04-19 19:40:28.465866,41863526108385,8,1,1,2019-04-20 02:52:26.892880,25918
11162626,2019-04-20 02:52:26.892880,41863526108385,3,1,1,2019-04-20 02:59:02.509230,396
11194917,2019-04-20 02:59:02.509230,41863526108385,3,1,1,2019-04-20 03:06:01.675788,419
11229465,2019-04-20 03:06:01.675788,41863526108385,3,1,1,2019-04-20 03:08:57.388160,176
11244486,2019-04-20 03:08:57.388160,41863526108385,3,1,1,2019-04-20 03:11:26.463903,149


## Lectura y limpieza de otros dataframes

Esto se hace despues pues preparar Auctions requiere de toda la memoria posible libre

In [9]:
events_dtypes = {
    'event_id': np.int64,
    'ref_hash': np.int64,
    'application_id': np.int64,
    'attributed': np.bool,
    'device_os_version': 'object',
    'device_brand': 'object',
    'device_model': 'object',
    'device_city': 'object',
    'session_user_agent': 'object',
    'trans_id': 'category',
    'user_agent': 'object',
    'event_uuid': 'object',
    'carrier': 'object',
    'kind': 'object',
    'device_os': 'object',
    'wifi': np.bool,
    'connection_type': 'object',
    #'ip_address': np.int64,
    #'device_language': 'category'
}
events_cols = list(events_dtypes.keys()) + ['date']
events = pd.read_csv(root_dir + 'events.csv.gzip', 
                     compression='gzip',
                     dtype=events_dtypes,
                     usecols=events_cols,
                     parse_dates=['date'])

events.rename({'date':'created'}, inplace=True, axis='columns')
events['n'] = 1
# Para hacer pruebas ocupando menos memoria se hace un sampleo aleatorio de la mitad del dataframe y se elimina el resto
if is_testing:
    events = events.sample(frac=.30)
    print("Is Testing, #records:", events['n'].sum())
else:
    print("Not Testing, #records:", events['n'].sum())

Not Testing, #records: 7744581


In [10]:
installs_dtypes = {
    'application_id': np.int32,
    'ref_hash': np.int64, 
    #'click_hash':'category',
    'attributed': 'category',
    'implicit': 'category',
    #'device_countrycode': 'object', 
    'device_brand': 'object',
    'device_model': 'object', 
    'session_user_agent': 'object', 
    'user_agent': 'object', 
    'event_uuid':'object',
    'kind': 'object',
    'wifi': 'object', 
    'trans_id': 'object', 
    #'ip_address':'object', 
    'device_language': 'object'
}
install_cols = list(installs_dtypes.keys()) + ['created']
installs = pd.read_csv(root_dir + 'installs.csv.gzip', 
                       compression='gzip', 
                       usecols=install_cols,
                       dtype= installs_dtypes,
                      parse_dates=['created'])

installs['n'] = 1
print('#records:', installs['n'].sum())

#records: 481511


In [11]:
clicks_dtypes = {
    'advertiser_id': np.int64, 
    'action_id': np.float, 
    'source_id': np.int8, 
    'latitude' : np.float64, 
    'longitude': np.float64, 
    'wifi_connection': np.bool, 
    'carrier_id': 'object', 
    'trans_id': 'object',
    'os_minor':'object', 
    'agent_device' : 'object', 
    'os_major': 'object', 
    'specs_brand': 'category', 
    'brand': 'object',
    'timeToClick': np.float64, 
    'touchX': 'object', 
    'touchY': 'object', 
    'ref_hash':np.int64,
    'created' : 'object'
}
clicks = pd.read_csv(root_dir + 'clicks.csv.gzip', 
                     compression='gzip',
                     usecols=list(clicks_dtypes.keys()) + ['created'],
                     dtype=clicks_dtypes,
                     parse_dates=['created'])
clicks['n'] = 1
print('#records:', clicks['n'].sum())

#records: 64296


In [12]:
installs = installs.loc[installs['ref_hash'].isin(unique_hashes['ref_hash'])]
installs.sort_values(by='created',inplace=True)
installs = pd.merge_asof(installs,windows,left_on='created',right_on='begin_date').drop('begin_date', axis='columns')

In [13]:
clicks = clicks.loc[clicks['ref_hash'].isin(unique_hashes['ref_hash'])]
clicks['created'] = clicks['created'].dt.tz_localize(None)
clicks.sort_values(by='created',inplace=True)
clicks = pd.merge_asof(clicks,windows,left_on='created',right_on='begin_date').drop('begin_date', axis='columns')

In [14]:
events = events.loc[events['ref_hash'].isin(unique_hashes['ref_hash'])]
events.sort_values(by='created',inplace=True)
events = pd.merge_asof(events,windows,left_on='created',right_on='begin_date').drop('begin_date', axis='columns')

### Arreglo de installs

In [15]:
installs['kind'] = installs['kind'].str.replace(' ','_')
installs['kind'] = installs['kind'].str.replace('af_app_open ','af_app_opened')
installs['kind'] = installs['kind'].str.replace('af_app_opend','af_app_opened')
installs['kind'] = installs['kind'].str.lower()

In [16]:
installs.isnull().sum()


created                    0
application_id             0
ref_hash                   0
attributed                 0
implicit                   0
device_brand          204813
device_model           26871
session_user_agent     14828
user_agent            150111
event_uuid            377704
kind                  377704
wifi                  186016
trans_id              472140
device_language        27552
n                          0
window_nr                  0
dtype: int64

In [17]:
for i in ['device_brand','device_model','session_user_agent','user_agent','kind','wifi','device_language']:
    installs[i] = installs[i].fillna('unknown')
installs['device_brand'] = installs['device_brand'].astype('category')
installs['device_model'] = installs['device_model'].astype('category')
installs['session_user_agent'] = installs['session_user_agent'].astype('category')
installs['user_agent'] = installs['user_agent'].astype('category')
installs['kind'] = installs['kind'].astype('category')
installs['wifi'] = installs['wifi'].astype('category')
installs['device_language'] = installs['device_language'].astype('category')

In [18]:
%%time
installs.sort_values(by=group+['created'],inplace=True)

installs['next_date'] = installs.groupby(group, as_index = False, sort=False)['created'].transform(lambda x: x.shift(-1))
installs = installs.loc[(~installs['next_date'].isnull())]
installs['secs_to_next'] = (installs['next_date'] - installs['created']).transform(lambda x: round(x.total_seconds()))

CPU times: user 4min 4s, sys: 4.56 s, total: 4min 8s
Wall time: 4min 4s


In [19]:
installs.head()


,created,application_id,ref_hash,attributed,implicit,device_brand,device_model,session_user_agent,user_agent,event_uuid,kind,wifi,trans_id,device_language,n,window_nr,next_date,secs_to_next
47506,2019-04-18 21:11:50.326,121,41863526108385,False,True,6.115025880051902e+18,1.6584170108376253e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 8.0.0; LDN-LX3...,2f8be0cc-297e-4c9c-a097-1096aa5824b5,open,false,NaN,6.977049253562486e+18,1,1,2019-04-18 21:11:51.966,2
47508,2019-04-18 21:11:51.966,121,41863526108385,False,False,6.115025880051902e+18,1.6584170108376253e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 8.0.0; LDN-LX3...,NaN,unknown,false,NaN,6.977049253562486e+18,1,1,2019-04-18 21:17:11.946,320
47813,2019-04-18 21:17:11.946,65,41863526108385,False,False,6.115025880051902e+18,1.6584170108376253e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 8.0.0; LDN-LX3...,NaN,unknown,false,NaN,6.977049253562486e+18,1,1,2019-04-18 21:17:16.531,5
362730,2019-04-24 20:39:14.142,145,448610188195811,False,True,2.987569314309514e+18,4.898310199028851e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 9; Redmi Note ...,7cbdf64a-a8cf-4a55-af4a-8b7181403feb,af_app_opened,false,NaN,6.977049253562486e+18,1,3,2019-04-24 20:39:55.008,41
208551,2019-04-21 22:31:11.662,65,475635010681369,False,True,4.533745055360187e+18,1.7261492746485394e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 8.1.0; Hisense...,56caed98-1785-43e7-8209-bbeb747f9268,af_app_opened,true,NaN,6.977049253562486e+18,1,2,2019-04-21 22:31:18.581,7


### Arreglo de Clicks

In [20]:
clicks.touchX = clicks.touchX.apply(lambda x: np.float64(x))
clicks.touchY = clicks.touchY.apply(lambda x: np.float64(x))
clicks['created'] = clicks['created'].dt.tz_localize(None)

In [21]:
for i in ['carrier_id','os_minor', 'agent_device', 'brand', 'os_major']:
    clicks[i] = clicks[i].fillna('unknown').astype('category')

In [22]:
clicks.head()

,advertiser_id,action_id,source_id,created,latitude,longitude,wifi_connection,carrier_id,trans_id,os_minor,agent_device,os_major,specs_brand,brand,timeToClick,touchX,touchY,ref_hash,n,window_nr
0,1,NaN,1,2019-04-12 00:00:01.981,1.714386,0.871408,False,1.0,OHtntFaZBW28OcZvxtGT0rLBLqmccrM,1.5176438893491397e+18,unknown,5.131615556736863e+18,3576558787748411622,0.0,19.296,0.952,0.070,3455656761384645032,1,NaN
1,2,NaN,1,2019-04-12 00:00:29.662,1.712736,0.869157,True,24.0,XZZOyg3ZOEFw9MFeBJWF7dNUl7Dl8AQ,3.575963029724781e+18,unknown,5.754947116114108e+18,3576558787748411622,0.0,625.859,0.474,11.684,2751776655120553475,1,NaN
2,1,NaN,0,2019-04-12 00:00:53.624,1.714547,0.871535,False,13.0,ptDi6mZepLORp2kxILbv_g5WxOVgqHI,1.2885781261232225e+18,unknown,3.908390200756879e+18,71913840936116953,unknown,29.908,0.130,0.179,6931142988092289392,1,NaN
3,1,NaN,1,2019-04-12 00:01:07.438,1.837799,0.829634,True,23.0,MhX94IlfeUwwWATXfHlLHKGPwSepPv4,2.2387361390161664e+18,unknown,3.581232574980917e+18,3576558787748411622,unknown,377.566,0.446,0.510,2114792698570967449,1,NaN
4,1,NaN,1,2019-04-12 00:01:08.535,1.733375,0.908130,True,1.0,BubcDEi6KKiYNUoN3XZmHioWNYna5N0,5.310344816890522e+18,unknown,3.581232574980917e+18,3576558787748411622,unknown,141.725,0.747,0.022,3949216411894914793,1,NaN


In [23]:
clicks.isnull().sum()


advertiser_id          0
action_id          63835
source_id              0
created                0
latitude               0
longitude              0
wifi_connection        0
carrier_id             0
trans_id               0
os_minor               0
agent_device           0
os_major               0
specs_brand            0
brand                  0
timeToClick        25885
touchX             20428
touchY             20428
ref_hash               0
n                      0
window_nr          19730
dtype: int64

In [24]:
clicks.head()

,advertiser_id,action_id,source_id,created,latitude,longitude,wifi_connection,carrier_id,trans_id,os_minor,agent_device,os_major,specs_brand,brand,timeToClick,touchX,touchY,ref_hash,n,window_nr
0,1,NaN,1,2019-04-12 00:00:01.981,1.714386,0.871408,False,1.0,OHtntFaZBW28OcZvxtGT0rLBLqmccrM,1.5176438893491397e+18,unknown,5.131615556736863e+18,3576558787748411622,0.0,19.296,0.952,0.070,3455656761384645032,1,NaN
1,2,NaN,1,2019-04-12 00:00:29.662,1.712736,0.869157,True,24.0,XZZOyg3ZOEFw9MFeBJWF7dNUl7Dl8AQ,3.575963029724781e+18,unknown,5.754947116114108e+18,3576558787748411622,0.0,625.859,0.474,11.684,2751776655120553475,1,NaN
2,1,NaN,0,2019-04-12 00:00:53.624,1.714547,0.871535,False,13.0,ptDi6mZepLORp2kxILbv_g5WxOVgqHI,1.2885781261232225e+18,unknown,3.908390200756879e+18,71913840936116953,unknown,29.908,0.130,0.179,6931142988092289392,1,NaN
3,1,NaN,1,2019-04-12 00:01:07.438,1.837799,0.829634,True,23.0,MhX94IlfeUwwWATXfHlLHKGPwSepPv4,2.2387361390161664e+18,unknown,3.581232574980917e+18,3576558787748411622,unknown,377.566,0.446,0.510,2114792698570967449,1,NaN
4,1,NaN,1,2019-04-12 00:01:08.535,1.733375,0.908130,True,1.0,BubcDEi6KKiYNUoN3XZmHioWNYna5N0,5.310344816890522e+18,unknown,3.581232574980917e+18,3576558787748411622,unknown,141.725,0.747,0.022,3949216411894914793,1,NaN


### Arreglo de Events

In [25]:
for i in ['device_os_version','device_brand','device_model','device_city','session_user_agent','user_agent','carrier','kind','device_os','connection_type']:
    events[i] = events[i].fillna('unknown').astype('category')

In [26]:
%%time

events.sort_values(by=group+['created'],inplace=True)

events['next_date'] = events.groupby(group, as_index = False, sort=False)['created'].transform(lambda x: x.shift(-1))
events = events.loc[(~events['next_date'].isnull())]
events['secs_to_next'] = (events['next_date'] - events['created']).transform(lambda x: round(x.total_seconds()))

CPU times: user 7min 26s, sys: 8.65 s, total: 7min 34s
Wall time: 7min 29s


In [27]:
events.isnull().sum()


created                     0
event_id                    0
ref_hash                    0
application_id              0
attributed                  0
device_os_version           0
device_brand                0
device_model                0
device_city                 0
session_user_agent          0
trans_id              7169599
user_agent                  0
event_uuid              25413
carrier                     0
kind                        0
device_os                   0
wifi                        0
connection_type             0
n                           0
window_nr                   0
next_date                   0
secs_to_next                0
dtype: int64

---
## 3. Armado de Features

A continuacion se comienzan a extraer los distintos features que formaran el set de entrenamiento. 
Se cruza cada ventana con los labels que se desean predecir. 

Para entrenar el set debe decidirse como utilizar los datos de las distintas ventanas. 

Se tomaran los datos en 2 ventanas diferentes y se validaran con dos ventanas diferentes, de esta forma no deberia haber problemas de solapamiento y se maximiza la cantidad de datos.

Para el set final puede armarse un set de labels con los valores de los ultimos 3 dias.

In [28]:
max_dates = pd.DataFrame({'w_max_date': [dt.datetime(2019,4,21),dt.datetime(2019,4,24),dt.datetime(2019,4,27)], 'window_nr' : [1,2,3]})

In [29]:
temp = unique_hashes
temp['window_nr'] = 1
devices = temp.copy()
temp['window_nr'] = 2
devices = devices.append(temp)
temp['window_nr'] = 3
devices = devices.append(temp)

In [30]:
devices['window_nr'].value_counts()

3    662110
2    662110
1    662110
Name: window_nr, dtype: int64

In [31]:
devices = devices.set_index(group)

In [32]:
sample = auctions.head(100)

In [33]:
sample.head()

,created,ref_hash,source_id,n,window_nr,next_date,secs_to_next
9146132,2019-04-19 19:40:28.465866,41863526108385,8,1,1,2019-04-20 02:52:26.892880,25918
11162626,2019-04-20 02:52:26.892880,41863526108385,3,1,1,2019-04-20 02:59:02.509230,396
11194917,2019-04-20 02:59:02.509230,41863526108385,3,1,1,2019-04-20 03:06:01.675788,419
11229465,2019-04-20 03:06:01.675788,41863526108385,3,1,1,2019-04-20 03:08:57.388160,176
11244486,2019-04-20 03:08:57.388160,41863526108385,3,1,1,2019-04-20 03:11:26.463903,149


### Tiempo promedio de arribos

Se desea saber el tiempo promedio entre arribos de los dispositivos a las encuestas.

In [34]:
time = auctions.groupby(group,sort=False)['secs_to_next'].agg(['mean','std','min','max'])\
               .rename({'mean':'auc_secs_to_next_mean','std':'auc_secs_to_next_std','min':'auc_secs_to_next_max','max':'auc_secs_to_next_min'}, axis='columns')\
               .transform(lambda x: round(x))

In [35]:
time.head()

auc_secs_to_next_mean  auc_secs_to_next_std  \
ref_hash        window_nr                                                
41863526108385  1                          935.0                4438.0   
135153013040192 1                          114.0                 166.0   
161514654074162 1                           25.0                  16.0   
186034136943920 1                        32918.0               49177.0   
                2                         3836.0                9041.0   

                           auc_secs_to_next_max  auc_secs_to_next_min  
ref_hash        window_nr                                              
41863526108385  1                             0                 25918  
135153013040192 1                             1                   480  
161514654074162 1                             3                    47  
186034136943920 1                            18                114850  
                2                             0                 35132

In [36]:
devices = devices.merge(time, how='left', left_index=True, right_index=True)

In [37]:
devices.head()

,,auc_secs_to_next_mean,auc_secs_to_next_std,auc_secs_to_next_max,auc_secs_to_next_min
ref_hash,window_nr,,,,
2564673204772915246,1,765.0,5431.0,0.0,56670.0
4441121667607578179,1,111.0,1555.0,0.0,38225.0
7721769811471055264,1,281.0,1179.0,0.0,15975.0
6416039086842158968,1,NaN,NaN,NaN,NaN
1258642015983312729,1,8365.0,28817.0,0.0,99871.0


In [38]:
time = installs.groupby(group,sort=False)['secs_to_next'].agg(['mean','std','min','max'])\
               .rename({'mean':'ins_secs_to_next_mean','std':'ins_secs_to_next_std','min':'ins_secs_to_next_max','max':'ins_secs_to_next_min'}, axis='columns')\
               .transform(lambda x: round(x))

In [39]:
time.head()

,,ins_secs_to_next_mean,ins_secs_to_next_std,ins_secs_to_next_max,ins_secs_to_next_min
ref_hash,window_nr,,,,
41863526108385,1,109.0,183.0,2,320
448610188195811,3,41.0,NaN,41,41
475635010681369,2,7.0,NaN,7,7
622102439689666,2,0.0,NaN,0,0
955103142671534,2,2.0,NaN,2,2


In [40]:
devices = devices.merge(time, how='left', left_index=True, right_index=True)

In [41]:
devices.head()

,,auc_secs_to_next_mean,auc_secs_to_next_std,auc_secs_to_next_max,auc_secs_to_next_min,ins_secs_to_next_mean,ins_secs_to_next_std,ins_secs_to_next_max,ins_secs_to_next_min
ref_hash,window_nr,,,,,,,,
2564673204772915246,1,765.0,5431.0,0.0,56670.0,NaN,NaN,NaN,NaN
4441121667607578179,1,111.0,1555.0,0.0,38225.0,NaN,NaN,NaN,NaN
7721769811471055264,1,281.0,1179.0,0.0,15975.0,NaN,NaN,NaN,NaN
6416039086842158968,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1258642015983312729,1,8365.0,28817.0,0.0,99871.0,NaN,NaN,NaN,NaN


In [42]:
time = events.groupby(group,sort=False)['secs_to_next'].agg(['mean','std','min','max'])\
               .rename({'mean':'eve_secs_to_next_mean','std':'eve_secs_to_next_std','min':'eve_secs_to_next_max','max':'eve_secs_to_next_min'}, axis='columns')\
               .transform(lambda x: round(x))

In [43]:
time.head()

eve_secs_to_next_mean  eve_secs_to_next_std  \
ref_hash       window_nr                                                
40621409780134 2                        19642.0               55367.0   
41863526108385 1                         1338.0                9668.0   
               2                           14.0                  21.0   
               3                           36.0                  68.0   
69039685746313 2                           57.0                  80.0   

                          eve_secs_to_next_max  eve_secs_to_next_min  
ref_hash       window_nr                                              
40621409780134 2                            15                156669  
41863526108385 1                             0                 86583  
               2                             0                    48  
               3                             0                   440  
69039685746313 2                             0                   148

In [44]:
devices = devices.merge(time, how='left', left_index=True, right_index=True)

In [45]:
devices.head()

,,auc_secs_to_next_mean,auc_secs_to_next_std,auc_secs_to_next_max,auc_secs_to_next_min,ins_secs_to_next_mean,ins_secs_to_next_std,ins_secs_to_next_max,ins_secs_to_next_min,eve_secs_to_next_mean,eve_secs_to_next_std,eve_secs_to_next_max,eve_secs_to_next_min
ref_hash,window_nr,,,,,,,,,,,,
2564673204772915246,1,765.0,5431.0,0.0,56670.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4441121667607578179,1,111.0,1555.0,0.0,38225.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7721769811471055264,1,281.0,1179.0,0.0,15975.0,NaN,NaN,NaN,NaN,33.0,49.0,0.0,266.0
6416039086842158968,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1258642015983312729,1,8365.0,28817.0,0.0,99871.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Tiempo desde ultima aparicion

In [46]:
%%time
time = auctions.groupby(group,sort=False)\
               .tail(1)\
               .merge(max_dates, on='window_nr', how='left')\
               .set_index(group)

time = (time['w_max_date'] - time['created']).transform(lambda x: round(x.total_seconds())).rename('secs_since_last_arrival')

CPU times: user 13.8 s, sys: 2.2 s, total: 16 s
Wall time: 16 s


In [47]:
time.head()

ref_hash         window_nr
41863526108385   1             70206
135153013040192  1             70659
161514654074162  1            248758
186034136943920  1             74393
                 2             37583
Name: secs_since_last_arrival, dtype: int64

In [48]:
devices = devices.merge(time, how='left', left_index=True, right_index=True)

In [49]:
devices.head()

,,auc_secs_to_next_mean,auc_secs_to_next_std,auc_secs_to_next_max,auc_secs_to_next_min,ins_secs_to_next_mean,ins_secs_to_next_std,ins_secs_to_next_max,ins_secs_to_next_min,eve_secs_to_next_mean,eve_secs_to_next_std,eve_secs_to_next_max,eve_secs_to_next_min,secs_since_last_arrival
ref_hash,window_nr,,,,,,,,,,,,,
2564673204772915246,1,765.0,5431.0,0.0,56670.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88625.0
4441121667607578179,1,111.0,1555.0,0.0,38225.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29052.0
7721769811471055264,1,281.0,1179.0,0.0,15975.0,NaN,NaN,NaN,NaN,33.0,49.0,0.0,266.0,333.0
6416039086842158968,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1258642015983312729,1,8365.0,28817.0,0.0,99871.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,121224.0


### Cantidad de apariciones en encuestas en la ventana

In [50]:
%%time
time = auctions.groupby(group ,sort=False)['n'].sum().rename('auctions_total')

CPU times: user 2.2 s, sys: 985 ms, total: 3.19 s
Wall time: 3.19 s


In [51]:
time.head()

ref_hash         window_nr
41863526108385   1            34
135153013040192  1             7
161514654074162  1             5
186034136943920  1             6
                 2            58
Name: auctions_total, dtype: int64

In [52]:
devices = devices.merge(time, how='left', left_index=True, right_index=True)

In [53]:
devices.head()

,,auc_secs_to_next_mean,auc_secs_to_next_std,auc_secs_to_next_max,auc_secs_to_next_min,ins_secs_to_next_mean,ins_secs_to_next_std,ins_secs_to_next_max,ins_secs_to_next_min,eve_secs_to_next_mean,eve_secs_to_next_std,eve_secs_to_next_max,eve_secs_to_next_min,secs_since_last_arrival,auctions_total
ref_hash,window_nr,,,,,,,,,,,,,,
2564673204772915246,1,765.0,5431.0,0.0,56670.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88625.0,216.0
4441121667607578179,1,111.0,1555.0,0.0,38225.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29052.0,1188.0
7721769811471055264,1,281.0,1179.0,0.0,15975.0,NaN,NaN,NaN,NaN,33.0,49.0,0.0,266.0,333.0,908.0
6416039086842158968,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1258642015983312729,1,8365.0,28817.0,0.0,99871.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,121224.0,12.0


### Cantidad de apariciones en encuestas en la ultima hora

In [54]:
%%time
time = auctions.merge(max_dates, on='window_nr', how='left')
time = time.loc[time['created'] >= (time['w_max_date'] - timedelta(hours=1))]\
           .groupby(group, sort=False)['n'].sum()\
           .rename('auctions_last_hour')

CPU times: user 14.4 s, sys: 9.7 s, total: 24.1 s
Wall time: 24.2 s


In [55]:
time.head()

ref_hash         window_nr
345999128501141  2             3
360710529886978  3             2
416301579449694  1             2
686608884458246  3            11
717556230663455  1            74
Name: auctions_last_hour, dtype: int64

In [56]:
devices = devices.merge(time, how='left', left_index=True, right_index=True)

In [57]:
devices.head()

,,auc_secs_to_next_mean,auc_secs_to_next_std,auc_secs_to_next_max,auc_secs_to_next_min,ins_secs_to_next_mean,ins_secs_to_next_std,ins_secs_to_next_max,ins_secs_to_next_min,eve_secs_to_next_mean,eve_secs_to_next_std,eve_secs_to_next_max,eve_secs_to_next_min,secs_since_last_arrival,auctions_total,auctions_last_hour
ref_hash,window_nr,,,,,,,,,,,,,,,
2564673204772915246,1,765.0,5431.0,0.0,56670.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88625.0,216.0,NaN
4441121667607578179,1,111.0,1555.0,0.0,38225.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29052.0,1188.0,NaN
7721769811471055264,1,281.0,1179.0,0.0,15975.0,NaN,NaN,NaN,NaN,33.0,49.0,0.0,266.0,333.0,908.0,23.0
6416039086842158968,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1258642015983312729,1,8365.0,28817.0,0.0,99871.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,121224.0,12.0,NaN


### Cantidad de apariciones en encuestas en el fin de semana

In [58]:
%%time
weekend = [20,21]
time = auctions.loc[(auctions['created'].dt.day).isin(weekend)]\
               .groupby(group,sort=False)['n'].sum()\
               .rename('amount_auctions_in_weekend')

CPU times: user 3.45 s, sys: 692 ms, total: 4.15 s
Wall time: 4.19 s


In [59]:
time.head()

ref_hash         window_nr
41863526108385   1            33
135153013040192  1             7
186034136943920  1             4
                 2            11
360710529886978  1             2
Name: amount_auctions_in_weekend, dtype: int64

In [60]:
devices = devices.merge(time, how='left', left_index=True, right_index=True)

In [61]:
devices.head()

,,auc_secs_to_next_mean,auc_secs_to_next_std,auc_secs_to_next_max,auc_secs_to_next_min,ins_secs_to_next_mean,ins_secs_to_next_std,ins_secs_to_next_max,ins_secs_to_next_min,eve_secs_to_next_mean,eve_secs_to_next_std,eve_secs_to_next_max,eve_secs_to_next_min,secs_since_last_arrival,auctions_total,auctions_last_hour,amount_auctions_in_weekend
ref_hash,window_nr,,,,,,,,,,,,,,,,
2564673204772915246,1,765.0,5431.0,0.0,56670.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88625.0,216.0,NaN,NaN
4441121667607578179,1,111.0,1555.0,0.0,38225.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29052.0,1188.0,NaN,640.0
7721769811471055264,1,281.0,1179.0,0.0,15975.0,NaN,NaN,NaN,NaN,33.0,49.0,0.0,266.0,333.0,908.0,23.0,218.0
6416039086842158968,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1258642015983312729,1,8365.0,28817.0,0.0,99871.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,121224.0,12.0,NaN,NaN


### Ultima aparicion es weekend

In [62]:
weekend = [20,21]
time = auctions.groupby(group, sort=False)\
               .tail(1)\
               .set_index(group)['created'].transform(lambda x: x.day in weekend)\
               .rename('is_last_weekend')

In [63]:
time.head()

ref_hash         window_nr
41863526108385   1             True
135153013040192  1             True
161514654074162  1            False
186034136943920  1             True
                 2            False
Name: is_last_weekend, dtype: bool

In [64]:
devices = devices.merge(time, how='left', left_index=True, right_index=True)

In [65]:
devices.head()

,,auc_secs_to_next_mean,auc_secs_to_next_std,auc_secs_to_next_max,auc_secs_to_next_min,ins_secs_to_next_mean,ins_secs_to_next_std,ins_secs_to_next_max,ins_secs_to_next_min,eve_secs_to_next_mean,eve_secs_to_next_std,eve_secs_to_next_max,eve_secs_to_next_min,secs_since_last_arrival,auctions_total,auctions_last_hour,amount_auctions_in_weekend,is_last_weekend
ref_hash,window_nr,,,,,,,,,,,,,,,,,
2564673204772915246,1,765.0,5431.0,0.0,56670.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88625.0,216.0,NaN,NaN,False
4441121667607578179,1,111.0,1555.0,0.0,38225.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29052.0,1188.0,NaN,640.0,True
7721769811471055264,1,281.0,1179.0,0.0,15975.0,NaN,NaN,NaN,NaN,33.0,49.0,0.0,266.0,333.0,908.0,23.0,218.0,True
6416039086842158968,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1258642015983312729,1,8365.0,28817.0,0.0,99871.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,121224.0,12.0,NaN,NaN,False


### Cantidad de sources distintos

In [66]:
temp = auctions.groupby(by = group +['source_id'], sort=False).agg({'n':'sum'}).reset_index()
temp['n'] = 1
temp = temp.groupby(by = group).agg({'n' : 'sum'})
temp.rename(columns = {'n': 'amount_dif_src' },inplace = True)

In [67]:
temp.head()

amount_dif_src
ref_hash        window_nr                
41863526108385  1                       3
135153013040192 1                       1
161514654074162 1                       2
186034136943920 1                       1
                2                       1

In [68]:
devices = devices.merge(temp, how = 'left', left_index=True, right_index=True)

In [69]:
devices.head()

,,auc_secs_to_next_mean,auc_secs_to_next_std,auc_secs_to_next_max,auc_secs_to_next_min,ins_secs_to_next_mean,ins_secs_to_next_std,ins_secs_to_next_max,ins_secs_to_next_min,eve_secs_to_next_mean,eve_secs_to_next_std,eve_secs_to_next_max,eve_secs_to_next_min,secs_since_last_arrival,auctions_total,auctions_last_hour,amount_auctions_in_weekend,is_last_weekend,amount_dif_src
ref_hash,window_nr,,,,,,,,,,,,,,,,,,
2564673204772915246,1,765.0,5431.0,0.0,56670.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88625.0,216.0,NaN,NaN,False,3.0
4441121667607578179,1,111.0,1555.0,0.0,38225.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29052.0,1188.0,NaN,640.0,True,5.0
7721769811471055264,1,281.0,1179.0,0.0,15975.0,NaN,NaN,NaN,NaN,33.0,49.0,0.0,266.0,333.0,908.0,23.0,218.0,True,3.0
6416039086842158968,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1258642015983312729,1,8365.0,28817.0,0.0,99871.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,121224.0,12.0,NaN,NaN,False,1.0


## Cantidad de eventos

In [70]:
temp = events.groupby(group, sort=False)\
             .agg({'n':'sum'})\
             .rename(columns = {'n': 'amount_events' })

In [71]:
temp.head()

amount_events
ref_hash       window_nr               
40621409780134 2                      8
41863526108385 1                     87
               2                      7
               3                     56
69039685746313 2                      3

In [72]:
devices = devices.merge(temp, how = 'left', left_index=True, right_index=True)

In [73]:
devices.head()

,,auc_secs_to_next_mean,auc_secs_to_next_std,auc_secs_to_next_max,auc_secs_to_next_min,ins_secs_to_next_mean,ins_secs_to_next_std,ins_secs_to_next_max,ins_secs_to_next_min,eve_secs_to_next_mean,eve_secs_to_next_std,eve_secs_to_next_max,eve_secs_to_next_min,secs_since_last_arrival,auctions_total,auctions_last_hour,amount_auctions_in_weekend,is_last_weekend,amount_dif_src,amount_events
ref_hash,window_nr,,,,,,,,,,,,,,,,,,,
2564673204772915246,1,765.0,5431.0,0.0,56670.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88625.0,216.0,NaN,NaN,False,3.0,NaN
4441121667607578179,1,111.0,1555.0,0.0,38225.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29052.0,1188.0,NaN,640.0,True,5.0,NaN
7721769811471055264,1,281.0,1179.0,0.0,15975.0,NaN,NaN,NaN,NaN,33.0,49.0,0.0,266.0,333.0,908.0,23.0,218.0,True,3.0,98.0
6416039086842158968,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1258642015983312729,1,8365.0,28817.0,0.0,99871.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,121224.0,12.0,NaN,NaN,False,1.0,NaN


### Tiempo desde ultimo evento

In [74]:
max_dates.head()

,w_max_date,window_nr
0,2019-04-21,1
1,2019-04-24,2
2,2019-04-27,3


In [75]:
temp = events.groupby(group,sort=False)\
             .agg({'created':'max'})\
             .reset_index()\
             .merge(max_dates, on='window_nr', how='left')\
             .set_index(group)
temp = (temp['w_max_date'] - temp['created']).transform(lambda x: round(x.total_seconds())).rename('secs_since_last_event')

In [76]:
temp.head()

ref_hash        window_nr
40621409780134  2             32304
41863526108385  1             66454
                2            250014
                3             65272
69039685746313  2             23241
Name: secs_since_last_event, dtype: int64

In [77]:
devices = devices.merge(temp, how = 'left', left_index=True, right_index=True)

In [78]:
devices.head()

,,auc_secs_to_next_mean,auc_secs_to_next_std,auc_secs_to_next_max,auc_secs_to_next_min,ins_secs_to_next_mean,ins_secs_to_next_std,ins_secs_to_next_max,ins_secs_to_next_min,eve_secs_to_next_mean,eve_secs_to_next_std,eve_secs_to_next_max,eve_secs_to_next_min,secs_since_last_arrival,auctions_total,auctions_last_hour,amount_auctions_in_weekend,is_last_weekend,amount_dif_src,amount_events,secs_since_last_event
ref_hash,window_nr,,,,,,,,,,,,,,,,,,,,
2564673204772915246,1,765.0,5431.0,0.0,56670.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88625.0,216.0,NaN,NaN,False,3.0,NaN,NaN
4441121667607578179,1,111.0,1555.0,0.0,38225.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29052.0,1188.0,NaN,640.0,True,5.0,NaN,NaN
7721769811471055264,1,281.0,1179.0,0.0,15975.0,NaN,NaN,NaN,NaN,33.0,49.0,0.0,266.0,333.0,908.0,23.0,218.0,True,3.0,98.0,196689.0
6416039086842158968,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1258642015983312729,1,8365.0,28817.0,0.0,99871.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,121224.0,12.0,NaN,NaN,False,1.0,NaN,NaN


### Alguno de los eventos fueron con wifi?

In [79]:
temp = events.groupby(group, sort=False).agg({'wifi':'sum'})
temp['wifi'] = temp['wifi'].astype(bool)

In [80]:
temp.head()

wifi
ref_hash       window_nr       
40621409780134 2          False
41863526108385 1          False
               2          False
               3          False
69039685746313 2           True

In [81]:
devices = devices.merge(temp, how = 'left', left_index=True, right_index=True)
#devices['wifi'].fillna(?)

In [82]:
devices.head()

,,auc_secs_to_next_mean,auc_secs_to_next_std,auc_secs_to_next_max,auc_secs_to_next_min,ins_secs_to_next_mean,ins_secs_to_next_std,ins_secs_to_next_max,ins_secs_to_next_min,eve_secs_to_next_mean,eve_secs_to_next_std,...,eve_secs_to_next_min,secs_since_last_arrival,auctions_total,auctions_last_hour,amount_auctions_in_weekend,is_last_weekend,amount_dif_src,amount_events,secs_since_last_event,wifi
ref_hash,window_nr,,,,,,,,,,,,,,,,,,,,,
2564673204772915246,1,765.0,5431.0,0.0,56670.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,88625.0,216.0,NaN,NaN,False,3.0,NaN,NaN,NaN
4441121667607578179,1,111.0,1555.0,0.0,38225.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,29052.0,1188.0,NaN,640.0,True,5.0,NaN,NaN,NaN
7721769811471055264,1,281.0,1179.0,0.0,15975.0,NaN,NaN,NaN,NaN,33.0,49.0,...,266.0,333.0,908.0,23.0,218.0,True,3.0,98.0,196689.0,False
6416039086842158968,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1258642015983312729,1,8365.0,28817.0,0.0,99871.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,121224.0,12.0,NaN,NaN,False,1.0,NaN,NaN,NaN


### Tiempo promedio de clicks

In [83]:
temp = clicks.groupby(by = group)\
             .agg({'timeToClick' : 'mean'})\
             .rename(columns = {'timeToClick' : 'timeToClick_mean'})

In [84]:
devices = devices.merge(temp, how = 'left', left_index=True, right_index=True)
#devices['timeToClick_mean'].fillna(?)

### Cantidad de advertiser que tiene cada ref_hash

In [85]:
temp = clicks.groupby(group + ['advertiser_id'],sort=False).agg({'n':'sum'}).reset_index()
temp['n'] = 1
temp = temp.groupby(group)\
           .agg({'n' : 'sum'})\
           .rename(columns = {'n': 'amount_dif_advertisers' })

In [86]:
temp.head()

,,amount_dif_advertisers
ref_hash,window_nr,
693609737448534,2.0,1
1461247282174365,1.0,1
2204225481747532,2.0,1
5470466329076030,1.0,1
5534899846690585,1.0,1


In [87]:
devices = devices.merge(temp, how = 'left', left_index=True, right_index=True)

### Cantidad de Installs

In [88]:
temp = installs.groupby(group,sort=False)['n'].sum().rename('amount_installs')

In [89]:
temp.head()

ref_hash         window_nr
41863526108385   1            3
448610188195811  3            1
475635010681369  2            1
622102439689666  2            1
955103142671534  2            1
Name: amount_installs, dtype: int64

In [90]:
devices = devices.merge(temp, how = 'left', left_index=True, right_index=True)

In [91]:
devices.head()

,,auc_secs_to_next_mean,auc_secs_to_next_std,auc_secs_to_next_max,auc_secs_to_next_min,ins_secs_to_next_mean,ins_secs_to_next_std,ins_secs_to_next_max,ins_secs_to_next_min,eve_secs_to_next_mean,eve_secs_to_next_std,...,auctions_last_hour,amount_auctions_in_weekend,is_last_weekend,amount_dif_src,amount_events,secs_since_last_event,wifi,timeToClick_mean,amount_dif_advertisers,amount_installs
ref_hash,window_nr,,,,,,,,,,,,,,,,,,,,,
2564673204772915246,1,765.0,5431.0,0.0,56670.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,False,3.0,NaN,NaN,NaN,NaN,NaN,NaN
4441121667607578179,1,111.0,1555.0,0.0,38225.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,640.0,True,5.0,NaN,NaN,NaN,NaN,NaN,NaN
7721769811471055264,1,281.0,1179.0,0.0,15975.0,NaN,NaN,NaN,NaN,33.0,49.0,...,23.0,218.0,True,3.0,98.0,196689.0,False,NaN,NaN,NaN
6416039086842158968,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1258642015983312729,1,8365.0,28817.0,0.0,99871.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,False,1.0,NaN,NaN,NaN,NaN,NaN,NaN


### Ultimo evento

In [92]:
temp = events.groupby(group,sort=False)\
             .tail(1)\
             .set_index(group)['event_id']\
             .rename('last_event')

In [93]:
temp.head()

ref_hash        window_nr
40621409780134  2            364
41863526108385  1              1
                2              2
                3              2
69039685746313  2            287
Name: last_event, dtype: int64

In [94]:
devices = devices.merge(temp, how = 'left', left_index=True, right_index=True)

In [95]:
devices.head()

,,auc_secs_to_next_mean,auc_secs_to_next_std,auc_secs_to_next_max,auc_secs_to_next_min,ins_secs_to_next_mean,ins_secs_to_next_std,ins_secs_to_next_max,ins_secs_to_next_min,eve_secs_to_next_mean,eve_secs_to_next_std,...,amount_auctions_in_weekend,is_last_weekend,amount_dif_src,amount_events,secs_since_last_event,wifi,timeToClick_mean,amount_dif_advertisers,amount_installs,last_event
ref_hash,window_nr,,,,,,,,,,,,,,,,,,,,,
2564673204772915246,1,765.0,5431.0,0.0,56670.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,False,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4441121667607578179,1,111.0,1555.0,0.0,38225.0,NaN,NaN,NaN,NaN,NaN,NaN,...,640.0,True,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7721769811471055264,1,281.0,1179.0,0.0,15975.0,NaN,NaN,NaN,NaN,33.0,49.0,...,218.0,True,3.0,98.0,196689.0,False,NaN,NaN,NaN,2.0
6416039086842158968,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1258642015983312729,1,8365.0,28817.0,0.0,99871.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,False,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Anteultimo evento

In [96]:
temp = events.groupby(group,sort=False)\
             .nth(-2)['event_id']\
             .rename('before_last_event')

In [97]:
temp.head()

ref_hash        window_nr
40621409780134  2              7
41863526108385  1              1
                2              2
                3              0
69039685746313  2            287
Name: before_last_event, dtype: int64

In [98]:
devices = devices.merge(temp, how = 'left', left_index=True, right_index=True)

In [99]:
devices.head()

,,auc_secs_to_next_mean,auc_secs_to_next_std,auc_secs_to_next_max,auc_secs_to_next_min,ins_secs_to_next_mean,ins_secs_to_next_std,ins_secs_to_next_max,ins_secs_to_next_min,eve_secs_to_next_mean,eve_secs_to_next_std,...,is_last_weekend,amount_dif_src,amount_events,secs_since_last_event,wifi,timeToClick_mean,amount_dif_advertisers,amount_installs,last_event,before_last_event
ref_hash,window_nr,,,,,,,,,,,,,,,,,,,,,
2564673204772915246,1,765.0,5431.0,0.0,56670.0,NaN,NaN,NaN,NaN,NaN,NaN,...,False,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4441121667607578179,1,111.0,1555.0,0.0,38225.0,NaN,NaN,NaN,NaN,NaN,NaN,...,True,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7721769811471055264,1,281.0,1179.0,0.0,15975.0,NaN,NaN,NaN,NaN,33.0,49.0,...,True,3.0,98.0,196689.0,False,NaN,NaN,NaN,2.0,2.0
6416039086842158968,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1258642015983312729,1,8365.0,28817.0,0.0,99871.0,NaN,NaN,NaN,NaN,NaN,NaN,...,False,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Ante anteultimo evento

In [100]:
temp = events.groupby(group,sort=False)\
             .nth(-3)['event_id']\
             .rename('before_before_last_event')

In [101]:
temp.head()

ref_hash        window_nr
40621409780134  2            363
41863526108385  1              1
                2              2
                3              1
69039685746313  2             15
Name: before_before_last_event, dtype: int64

In [102]:
devices = devices.merge(temp, how = 'left', left_index=True, right_index=True)

In [103]:
devices.head()

,,auc_secs_to_next_mean,auc_secs_to_next_std,auc_secs_to_next_max,auc_secs_to_next_min,ins_secs_to_next_mean,ins_secs_to_next_std,ins_secs_to_next_max,ins_secs_to_next_min,eve_secs_to_next_mean,eve_secs_to_next_std,...,amount_dif_src,amount_events,secs_since_last_event,wifi,timeToClick_mean,amount_dif_advertisers,amount_installs,last_event,before_last_event,before_before_last_event
ref_hash,window_nr,,,,,,,,,,,,,,,,,,,,,
2564673204772915246,1,765.0,5431.0,0.0,56670.0,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4441121667607578179,1,111.0,1555.0,0.0,38225.0,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7721769811471055264,1,281.0,1179.0,0.0,15975.0,NaN,NaN,NaN,NaN,33.0,49.0,...,3.0,98.0,196689.0,False,NaN,NaN,NaN,2.0,2.0,2.0
6416039086842158968,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1258642015983312729,1,8365.0,28817.0,0.0,99871.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


---
## 4. Guardo Features 
Aplico arreglo: 
* Si el dispositivo no aparecio en la ventana, se elimina la fila
* Si el dispositivo no tiene valores en los eventos de count agrego count 0

In [104]:
#Chequear casos que queden con medias en null
devices = devices.dropna(how='all')

In [105]:
devices.isnull().sum()

auc_secs_to_next_mean         316502
auc_secs_to_next_std          316502
auc_secs_to_next_max          316502
auc_secs_to_next_min          316502
ins_secs_to_next_mean         866415
ins_secs_to_next_std          924779
ins_secs_to_next_max          866415
ins_secs_to_next_min          866415
eve_secs_to_next_mean         505160
eve_secs_to_next_std          593715
eve_secs_to_next_max          505160
eve_secs_to_next_min          505160
secs_since_last_arrival       316502
auctions_total                316502
auctions_last_hour            868880
amount_auctions_in_weekend    655515
is_last_weekend               316502
amount_dif_src                316502
amount_events                 505160
secs_since_last_event         505160
wifi                          505160
timeToClick_mean              913440
amount_dif_advertisers        902618
amount_installs               866415
last_event                    505160
before_last_event             593715
before_before_last_event      647379
d

In [106]:
devices.dtypes

auc_secs_to_next_mean         float64
auc_secs_to_next_std          float64
auc_secs_to_next_max          float64
auc_secs_to_next_min          float64
ins_secs_to_next_mean         float64
ins_secs_to_next_std          float64
ins_secs_to_next_max          float64
ins_secs_to_next_min          float64
eve_secs_to_next_mean         float64
eve_secs_to_next_std          float64
eve_secs_to_next_max          float64
eve_secs_to_next_min          float64
secs_since_last_arrival       float64
auctions_total                float64
auctions_last_hour            float64
amount_auctions_in_weekend    float64
is_last_weekend                object
amount_dif_src                float64
amount_events                 float64
secs_since_last_event         float64
wifi                           object
timeToClick_mean              float64
amount_dif_advertisers        float64
amount_installs               float64
last_event                    float64
before_last_event             float64
before_befor

In [107]:
#Fix means
means = [x for x in devices.columns if 'mean' in x]
for i in means:
    print("Filling ",i, " with window mean")
    devices[i] = devices.groupby('window_nr')[i].transform(lambda x: x.fillna(round(x.mean())))

#Fix Strings
obj = list(devices.dtypes[devices.dtypes == 'object'].index)
for i in obj:
    print("Filling ",i, " with False")
    devices[i] = devices[i].fillna(False)

print('Filling the rest with 0')
devices = devices.fillna(0)

Filling  auc_secs_to_next_mean  with window mean
Filling  ins_secs_to_next_mean  with window mean
Filling  eve_secs_to_next_mean  with window mean
Filling  timeToClick_mean  with window mean
Filling  is_last_weekend  with False
Filling  wifi  with False
Filling the rest with 0


In [108]:
devices.isnull().sum()

auc_secs_to_next_mean         0
auc_secs_to_next_std          0
auc_secs_to_next_max          0
auc_secs_to_next_min          0
ins_secs_to_next_mean         0
ins_secs_to_next_std          0
ins_secs_to_next_max          0
ins_secs_to_next_min          0
eve_secs_to_next_mean         0
eve_secs_to_next_std          0
eve_secs_to_next_max          0
eve_secs_to_next_min          0
secs_since_last_arrival       0
auctions_total                0
auctions_last_hour            0
amount_auctions_in_weekend    0
is_last_weekend               0
amount_dif_src                0
amount_events                 0
secs_since_last_event         0
wifi                          0
timeToClick_mean              0
amount_dif_advertisers        0
amount_installs               0
last_event                    0
before_last_event             0
before_before_last_event      0
dtype: int64

### Agrego Clustering con KMeans 

In [109]:
devices['kmeans20'] = KMeans(n_clusters=20, random_state=0, n_jobs=cores).fit(devices).labels_
devices['kmeans50'] = KMeans(n_clusters=50, random_state=0, n_jobs=cores).fit(devices).labels_
#devices['kmeans100'] = KMeans(n_clusters=100, random_state=0, n_jobs=cores).fit(devices).labels_

### Agrego Clustering con Spectral 

In [110]:
%%time
# No le da la memoria
#devices['clust20'] = SpectralClustering(n_clusters=20, assign_labels="discretize", random_state=0,n_jobs=cores).fit(devices).labels_

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 8.11 µs


In [111]:
devices.to_csv('training_set.csv',header=True,index=True)